# Trading Report — Italian Equity Signals

Reads `data/results/it/analysis_results.parquet` (produced by `analyze_stock.py`) and surfaces:
- **Actionable trades**: Enter / Exit signals from the last 4 trading days
- **Per-signal status**: latest active signals per strategy
- **Full dashboard**: flat DataFrame exported to Excel

> Prerequisites: run `analyze_stock.py` first so stop-loss columns are present.

## 1. Setup

In [ ]:
import logging
import sys
from pathlib import Path

import pandas as pd

# UTF-8 so algoshort unicode symbols (✓ etc.) render correctly on Windows
sys.stdout.reconfigure(encoding="utf-8")
sys.stderr.reconfigure(encoding="utf-8")

from algoshort.trading_summary import (
    get_multi_symbol_summary,
    print_multi_symbol_summary,
)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
)
log = logging.getLogger(__name__)

# Suppress internal HOLD LONG / HOLD SHORT noise from algoshort
logging.getLogger("algoshort").setLevel(logging.WARNING)
logging.getLogger("algoshort.trading_summary").setLevel(logging.WARNING)

In [ ]:
# ── Paths ──────────────────────────────────────────────────────────────────
RESULTS_PATH   = Path("./data/results/it/analysis_results.parquet")
DASHBOARD_PATH = Path("./data/results/it/trading_dashboard.xlsx")

# Position-sizing column suffixes written by algoshort.position_sizing
_PS_SUFFIXES: dict[str, str] = {
    "equal":    "_shares_equal",
    "constant": "_shares_constant",
    "concave":  "_shares_concave",
    "convex":   "_shares_convex",
}

# How many recent trading days to include in the actionable summary
LOOKBACK_DAYS = 4
LOOKBACK_BARS = 5  # bars passed to get_multi_symbol_summary

## 2. Load Results

In [ ]:
def load_results(path: Path) -> dict[str, pd.DataFrame]:
    """Load the combined parquet and split it into per-symbol DataFrames."""
    log.info("Loading results from %s", path)
    df = pd.read_parquet(path)
    if "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"])
    data_dict = {symbol: grp.copy() for symbol, grp in df.groupby("symbol")}
    log.info("Loaded %d symbols.", len(data_dict))
    return data_dict


data_dict = load_results(RESULTS_PATH)

first_df = next(iter(data_dict.values()))
print(f"Symbols loaded : {len(data_dict)}")
print(f"Rows per symbol: ~{len(first_df)}")
print(f"Date range     : {first_df['date'].min().date()} → {first_df['date'].max().date()}")

## 3. Detect Signal Columns

In [ ]:
def detect_signal_columns(df: pd.DataFrame) -> list[str]:
    """
    Return column names that have a matching <col>_stop_loss counterpart.
    This is the reliable marker distinguishing primary trading signals from
    derived metrics (returns, cumulative P&L, etc.).
    """
    cols = set(df.columns)
    return sorted(c for c in cols if f"{c}_stop_loss" in cols)


def build_position_cols(signal: str, available: set[str]) -> dict[str, str] | None:
    """Build position_cols mapping, keeping only columns that actually exist."""
    mapping = {
        label: f"{signal}{suffix}"
        for label, suffix in _PS_SUFFIXES.items()
        if f"{signal}{suffix}" in available
    }
    return mapping or None


available_cols  = set(first_df.columns)
signal_columns  = detect_signal_columns(first_df)

if not signal_columns:
    stop_loss_present = sorted(c for c in available_cols if c.endswith("_stop_loss"))
    raise RuntimeError(
        f"No signal columns detected (need matching <col>_stop_loss).\n"
        f"stop_loss cols found : {stop_loss_present}\n"
        f"Re-run analyze_stock.py to regenerate results with stop-loss columns."
    )

print(f"Detected {len(signal_columns)} signal columns:")
for col in signal_columns:
    has_sizing = any(f"{col}{s}" in available_cols for s in _PS_SUFFIXES.values())
    print(f"  {col}" + ("  [+ position sizing]" if has_sizing else ""))

## 4. Build Summaries

In [ ]:
def summaries_to_dataframe(summaries: list[dict], signal: str) -> pd.DataFrame:
    """Flatten get_multi_symbol_summary output into one row per symbol."""
    rows = []
    for s in summaries:
        if "error" in s:
            rows.append({"signal": signal, "ticker": s["ticker"], "error": s["error"]})
            continue
        row = {
            "signal":         signal,
            "ticker":         s["ticker"],
            "last_date":      pd.to_datetime(s["last_date"]),
            "price":          s["current_price"],
            "position":       s["position_direction"],
            "action":         s["trade_action"],
            "signal_changed": s["signal_changed"],
            "stop_loss":      s["stop_loss"],
            "risk_pct":       s["risk_pct"],
        }
        for label, shares in s.get("position_sizes", {}).items():
            row[f"shares_{label}"] = shares
        rows.append(row)
    return pd.DataFrame(rows)


all_summaries: dict[str, list[dict]] = {}
all_dashboards: list[pd.DataFrame] = []

for signal in signal_columns:
    position_cols = build_position_cols(signal, available_cols)
    summaries = get_multi_symbol_summary(
        data_dict=data_dict,
        signal_col=signal,
        stop_loss_col=f"{signal}_stop_loss",
        close_col="close",
        position_cols=position_cols,
        lookback=LOOKBACK_BARS,
    )
    all_summaries[signal] = summaries
    all_dashboards.append(summaries_to_dataframe(summaries, signal))
    log.info("Signal '%s' done: %d summaries.", signal, len(summaries))

print("Done.")

## 5. Full Dashboard

In [ ]:
dashboard = pd.concat(all_dashboards, ignore_index=True)

DASHBOARD_PATH.parent.mkdir(parents=True, exist_ok=True)
dashboard.to_excel(DASHBOARD_PATH, index=False)
log.info("Dashboard saved → %s  (%d rows)", DASHBOARD_PATH, len(dashboard))

print(f"Dashboard shape: {dashboard.shape}")
dashboard.head()

## 6. Actionable Trades — Last 4 Trading Days

In [ ]:
unique_dates = sorted(dashboard["last_date"].unique(), reverse=True)
latest_date  = unique_dates[0]
cutoff_index = min(LOOKBACK_DAYS - 1, len(unique_dates) - 1)
four_days_ago = unique_dates[cutoff_index]

is_recent = dashboard["last_date"] >= four_days_ago
is_action = dashboard["action"].str.contains("Enter|Exit", case=False, na=False)

actionable = (
    dashboard[is_recent & is_action]
    .sort_values(by="last_date", ascending=False)
    .copy()
)

print(f"Actionable trades: {four_days_ago.date()} → {latest_date.date()}")
print(f"Found {len(actionable)} Enter/Exit signals.\n")

display_cols = ["last_date", "signal", "ticker", "price", "action", "stop_loss", "risk_pct"]
actionable[display_cols] if not actionable.empty else print("No Enter/Exit signals in the last 4 trading days.")

## 7. Per-Signal Status — Latest Day

In [ ]:
print(f"Active signals on {latest_date.date()}\n")

for signal in signal_columns:
    active = [
        s for s in all_summaries[signal]
        if pd.to_datetime(s.get("last_date")) == latest_date
        and any(act in str(s.get("trade_action", "")) for act in ["Enter", "Exit"])
    ]
    if active:
        print(f"\n>>> [ {signal} ]")
        print("-" * 35)
        print_multi_symbol_summary(active, detailed=False)

## 8. Quick Exploration Helpers

In [ ]:
# ── Distribution of actions across all signals ──────────────────────────────
dashboard.groupby(["signal", "action"]).size().unstack(fill_value=0)

In [ ]:
# ── Filter to a specific ticker across all signals ───────────────────────────
TICKER = "AVIO.MI"  # ← change me
dashboard[dashboard["ticker"] == TICKER]

In [ ]:
# ── Filter to a specific signal ───────────────────────────────────────────────
SIGNAL = signal_columns[0]  # ← change me
sig_df = dashboard[dashboard["signal"] == SIGNAL].sort_values("action")
sig_df[["ticker", "price", "position", "action", "stop_loss", "risk_pct"]]